In [3]:
%pip install langchain
%pip install serpapi
%pip install langchain-google-genai
%pip install google-search-results

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from serpapi.google_search import GoogleSearch

def fetch_hotels(query: str):
    """
    query: e.g. "Calangute Beach, Goa"
    Returns: list of hotel dicts from SerpAPI Google Hotels
    """

    params = {
        "engine":         "google_hotels",
        "q":              query,
        "api_key":        "Serp_API_KEY",
        "currency": "INR",
        "check_in_date":  "2025-05-24",
        "check_out_date": "2025-05-28",
    }
    search = GoogleSearch(params)
    data = search.get_dict()
    return data.get("properties", [])   # hotels live under "properties"

res = fetch_hotels("Calangute Beach, Goa")
print(res)



In [18]:
def organize_with_gemini(hotels: list, user_pref: str) -> str:
    """
    hotels: raw list from SerpAPI
    user_pref: e.g. "show me hotels near Calangute Beach Goa under ₹5000"
    Returns: clean bullet‑list text including name, price, location, and booking link
    """

    blob = ""
    for h in hotels:
        name    = h.get("name", "Unknown")
        price   = h.get("rate_per_night", {}).get("lowest", "Not specified")
        rating  = h.get("overall_rating", "Not specified")
        nearby = h.get("nearby_places", "Not specified")
        link    = h.get("hotel_reviews_link") or h.get("link") or "Not available"
        blob += f"Hotel Name: {name} — Price: {price} — Rating: {rating } — Nearby Place: {nearby} — Link: {link}\n"

    prompt = ChatPromptTemplate.from_template("""
You are a travel assistant. The user said:
  {user_pref}

From this list of hotels:
{hotel_blob}

You are a helpfull ai assitant. Filter to only those that satisfy the user’s request (e.g. price limits),
and output a **bullet‑point list** (at most 10) where each item shows:
- Hotel Name
- Price
- Rating
- Nearby place
- Booking Link

Do not return JSON, just bullet points.
""")
    
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-001",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
)
    chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
    return chain.run(user_pref=user_pref, hotel_blob=blob)

In [ ]:
if __name__ == "__main__":
    
    user_pref = input("Your hotel request (e.g. 'hotels near Calangute Beach Goa under ₹5000'): ")

    location = user_pref.split(" under ")[0].replace("show me hotels near ", "").strip()


    hotels = fetch_hotels(location)
    if not hotels:
        print("No hotels found.")
    else:
        result = organize_with_gemini(hotels, user_pref)
        print("\nHere are the matching hotels:\n")
        print(result)